<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#WHO-Covid-data-gathering-and-Analysis" data-toc-modified-id="WHO-Covid-data-gathering-and-Analysis-1">WHO Covid data gathering and Analysis</a></span><ul class="toc-item"><li><span><a href="#Set-up-libraries" data-toc-modified-id="Set-up-libraries-1.1">Set up libraries</a></span></li><li><span><a href="#Set-up-helper-functions" data-toc-modified-id="Set-up-helper-functions-1.2">Set up helper functions</a></span><ul class="toc-item"><li><span><a href="#FInd-what-is-at-a-certain-row-and-column" data-toc-modified-id="FInd-what-is-at-a-certain-row-and-column-1.2.1">FInd what is at a certain row and column</a></span></li><li><span><a href="#Task-2-we-will-import-all-the-reports-since-May-2020-and-implement-a-solution-that-will-combine-data-from-new-PDF-files-as-they-are-been-published-on-the-WHO-website." data-toc-modified-id="Task-2-we-will-import-all-the-reports-since-May-2020-and-implement-a-solution-that-will-combine-data-from-new-PDF-files-as-they-are-been-published-on-the-WHO-website.-1.2.2">Task 2 we will import all the reports since May 2020 and implement a solution that will combine data from new PDF files as they are been published on the WHO website.</a></span><ul class="toc-item"><li><span><a href="#Step-1" data-toc-modified-id="Step-1-1.2.2.1">Step 1</a></span></li></ul></li><li><span><a href="#Using-the--URL-List-Concat-together-reports-for-all-dates-from-July-15th-through-today" data-toc-modified-id="Using-the--URL-List-Concat-together-reports-for-all-dates-from-July-15th-through-today-1.2.3">Using the  URL List Concat together reports for all dates from July 15th through today</a></span></li></ul></li><li><span><a href="#Create-a-calendar-with-dates-from-January-up-until-now.-Add-columns-with-Year,-Week-Number,-Start-of-Weekdate-and-end-of-week-date" data-toc-modified-id="Create-a-calendar-with-dates-from-January-up-until-now.-Add-columns-with-Year,-Week-Number,-Start-of-Weekdate-and-end-of-week-date-1.3">Create a calendar with dates from January up until now. Add columns with Year, Week Number, Start of Weekdate and end of week date</a></span></li><li><span><a href="#Using-Camelot-and-Excalibur-for-PDF" data-toc-modified-id="Using-Camelot-and-Excalibur-for-PDF-1.4">Using Camelot and Excalibur for PDF</a></span></li></ul></li></ul></div>

# WHO Covid data gathering and Analysis
<a href = 'https://datachant.com/2020/08/14/import-who-covid-19-data-from-pdf-2/'> Gil Raviv Blogpost tutorials on WHO Covid Data analysis </a>

## Set up libraries

In [197]:
import pandas as pd
import datetime
import re

import ghostscript
import  camelot

#import tabula as tby



#References
#https://pypi.org/project/tabula-py/
#https://www.youtube.com/watch?v=x6AOKcPar2M

#Faced issues with Tabula-Py and hence using Camelot


#Rejection of Tabula-py and choose Camelot
#Tabula-py was not reading some pages of certain URLS  
# Tabula-py was not reading in diacritical marks of countries correctly as in Cote De Ivoire

# pip install tk
# pip install ghostscript
# pip install camelot-py[cv]
# pip install excalibur-py



In [124]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 500)

## Set up helper functions

In [198]:
def dfFromURL(urlString):
    print('Extracting DFs for ', urlString)
    capture_df_list = []
    for oneTable in tables:
        df = oneTable.df
        if df.loc[:0, 0].str.contains('Table 1. Countries, territories or areas with').any():
            df = df[1:]
            capture_df_list.append(df)
        elif df.loc[:0, 0].str.contains('Reporting Country/Territory/Area').any():
            capture_df_list.append(df) 
    return capture_df_list



In [199]:
def cleanReportForURL(urlString):
    pattern = re.compile(r'[0-9]{8}')
    dateMatch = pattern.search(urlString)
    dateInURL = dateMatch.group()

    try:
        df_list= dfFromURL(urlString)
    except:
        return None
    
    # Concat all the units of a dataframe together
    df_all = pd.concat(df_list)
    df_all.info()

    df_all.reset_index(drop=True, inplace=True)
    #Set the columns
    df_all.columns = ['Country', 'Total confirmed cases','Total confirmed new cases', 'Total deaths', 'Total new deaths', 'Transmission classification', 'Days since last reported case']

    
    #Remove repeated heading and second row of headings
    df_all = df_all[~(df_all['Country'] == 'Reporting Country/Territory/Area')]
    df_all = df_all[~(df_all['Total confirmed cases'] == 'cases')]
    
    for col in ['Total confirmed cases', 'Total confirmed new cases',  'Total deaths', 'Total new deaths']:
        df_all[col].replace('', np.NaN, inplace=True)
        
    # Remove Continent names 
    df_all = df_all.dropna(subset = ['Total confirmed cases', 'Total confirmed new cases', 'Total deaths'])
    df_all = df_all[~(df_all['Country'].isin(['Grand total', 'Subtotal for all regions', 'Other*']))]

    # When facing challenges in changing data type, check if you have strings in any of the columns or rows
    for col in ['Total confirmed cases', 'Total confirmed new cases',  'Total deaths', 'Total new deaths']:
        df_all[col] = df_all[col].str.replace(' ','')
        df_all[col] = df_all[col].fillna(0)
        df_all[col].astype(str).astype(int)
   
    df_all['Report date'] = dateInURL
    df_all.set_index('Country',inplace=True)
    return df_all



### FInd what is at a certain row and column

In [166]:
df.at[df.index[df['Country'] == 'Africa'].values[0], 'Total confirmed cases']


''

### Task 2 we will import all the reports since May 2020 and implement a solution that will combine data from new PDF files as they are been published on the WHO website.




#### Step 1
<code>
Our objective is to create a list of filenames starting from:
20200715-covid-19-sitrep-177.pdf
And ending with:
20200811-covid-19-sitrep-204.pdf
</code>    

In [200]:
def createURLList():
    urlListStart = [r'https://www.who.int/docs/default-source/coronaviruse/situation-reports/'+date.strftime("%Y%m%d") + '-covid-19-sitrep-' for date in pd.date_range('20200715', pd.to_datetime('today'), freq='D')]
    urlList =  [url+str(i+177)+'.pdf' for i , url in enumerate(urlListStart)]
    return urlList

urlList = createURLList() 
urlList[:5]

['https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200715-covid-19-sitrep-177.pdf',
 'https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200716-covid-19-sitrep-178.pdf',
 'https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200717-covid-19-sitrep-179.pdf',
 'https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200718-covid-19-sitrep-180.pdf',
 'https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200719-covid-19-sitrep-181.pdf']

### Using the  URL List Concat together reports for all dates from July 15th through today

In [195]:
list_of_df = []

In [196]:
for eachURL in urlList:
    df = cleanReportForURL(eachURL)
    if df is not None:
        list_of_df.append(df)
    


Extracting DFs for  https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200715-covid-19-sitrep-177.pdf
<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1 to 21
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       252 non-null    object
 1   1       252 non-null    object
 2   2       252 non-null    object
 3   3       252 non-null    object
 4   4       252 non-null    object
 5   5       252 non-null    object
 6   6       252 non-null    object
dtypes: object(7)
memory usage: 15.8+ KB
Extracting DFs for  https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200716-covid-19-sitrep-178.pdf
<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1 to 21
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       252 non-null    object
 1   1       252 non-null    object
 2   2       252 non-null    object
 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1 to 21
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       252 non-null    object
 1   1       252 non-null    object
 2   2       252 non-null    object
 3   3       252 non-null    object
 4   4       252 non-null    object
 5   5       252 non-null    object
 6   6       252 non-null    object
dtypes: object(7)
memory usage: 15.8+ KB
Extracting DFs for  https://www.who.int/docs/default-source/coronaviruse/situation-reports/20200803-covid-19-sitrep-196.pdf
<class 'pandas.core.frame.DataFrame'>
Int64Index: 252 entries, 1 to 21
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       252 non-null    object
 1   1       252 non-null    object
 2   2       252 non-null    object
 3   3       252 non-null    object
 4   4       252 non-null    object
 5   5       252 non-null    object
 6   6       252 

In [201]:
list_of_df_copy = list_of_df.copy()

In [202]:
final_df = pd.concat(list_of_df)

In [203]:
final_df.to_csv('FinalDF.csv')

## Create a calendar with dates from January up until now. Add columns with Year, Week Number, Start of Weekdate and end of week date

In [57]:
list_of_df_copy

[                         Total confirmed cases Total confirmed new cases  \
 Country                                                                    
 South Africa                            298292                     10496   
 Nigeria                                  33616                       463   
 Ghana                                    24988                         0   
 Algeria                                  20216                       527   
 Cameroon                                 15173                         0   
 ...                                        ...                       ...   
 Papua New Guinea                            11                         0   
 Guam                                       305                         1   
 French Polynesia                            62                         0   
 Northern Mariana Islands                    36                         3   
 New Caledonia                               22                         1   

In [159]:
calendar_df = pd.DataFrame(pd.date_range('20200715', pd.to_datetime('today'), freq='D'), columns=['Date'])
calendar_df['Year'] = pd.DatetimeIndex(calendar_df['Date']).year
calendar_df['WeekNumber'] = calendar_df['Date'].dt.isocalendar().week
#Get the weekday of the date in the table and go back those many days for start of week
calendar_df['Start of Week'] = calendar_df['Date'] - pd.to_timedelta(calendar_df['Date'].dt.weekday, unit='D')
#Add 6 to the start date of the week for end of the week
calendar_df['End of week']= calendar_df['Start of Week']  + pd.to_timedelta(6, unit='D')
calendar_df


,Date,Year,WeekNumber,Start of Week,End of week
0,2020-07-15,2020,29,2020-07-13,2020-07-19
1,2020-07-16,2020,29,2020-07-13,2020-07-19
2,2020-07-17,2020,29,2020-07-13,2020-07-19
3,2020-07-18,2020,29,2020-07-13,2020-07-19
4,2020-07-19,2020,29,2020-07-13,2020-07-19
5,2020-07-20,2020,30,2020-07-20,2020-07-26
6,2020-07-21,2020,30,2020-07-20,2020-07-26
7,2020-07-22,2020,30,2020-07-20,2020-07-26
8,2020-07-23,2020,30,2020-07-20,2020-07-26
9,2020-07-24,2020,30,2020-07-20,2020-07-26


## Using Camelot and Excalibur for PDF

In [94]:
tables[1].df

,0,1,2,3,4,5,6
0,"Table 1. Countries, territories or areas with ...",,,,,,
1,Reporting Country/Territory/Area,Total confirmed*,Total confirmed*,Total deaths,Total new deaths,Transmission,Days since last
2,,cases,new cases,,,classificationi,reported case
3,Africa,,,,,,
4,South Africa,421 996,13 944,6 343,250,Community transmission,0
5,Nigeria,39 539,591,845,12,Community transmission,0
6,Ghana,31 057,1 385,161,8,Community transmission,0
7,Algeria,26 159,675,1 136,12,Community transmission,0
8,Cameroon,16 708,186,385,3,Community transmission,0
9,Kenya,16 268,667,274,11,Community transmission,0


In [72]:
tables = camelot.read_pdf('foo.pdf', flavor='stream' )

In [73]:
tables.pr

<TableList n=1>